In [1]:
import os
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/saikishan2452003/Data-Science-Project.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="saikishan2452003"
os.environ["MLFLOW_TRACKING_PASSWORD"]="4de90a054bb53da31cb7b66ebb9dbbab0d2d208f"

In [2]:
import os 
%pwd

'e:\\MLOPS\\Data-Science-Project\\reseach'

In [3]:
os.chdir("../")
%pwd

'e:\\MLOPS\\Data-Science-Project'

In [4]:
from dataclasses import dataclass
from pathlib import Path


@dataclass
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str


In [5]:
from src.datascience.constants import *
from src.datascience.utils.common import read_yaml, create_directories,save_json

PROJECT_ROOT: e:\MLOPS\Data-Science-Project
CONFIG exists: True


In [6]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config=self.config.model_evaluation
        params=self.params.ElasticNet
        schema=self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config=ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path = config.model_path,
            all_params=params,
            metric_file_name = config.metric_file_name,
            target_column = schema.name,
            mlflow_uri="https://dagshub.com/saikishan2452003/Data-Science-Project.mlflow"


        )
        return model_evaluation_config


In [7]:
import os
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib

e:\MLOPS\Data-Science-Project\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    def eval_metrics(self,actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2
    
    def log_into_mlflow(self):

        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[[self.config.target_column]]


        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():

            predicted_qualities = model.predict(test_x)

            (rmse, mae, r2) = self.eval_metrics(test_y, predicted_qualities)
            
            # Saving metrics as local
            scores = {"rmse": rmse, "mae": mae, "r2": r2}
            save_json(path=Path(self.config.metric_file_name), data=scores)

            mlflow.log_params(self.config.all_params)

            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("r2", r2)
            mlflow.log_metric("mae", mae)


            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.sklearn.log_model(model, "model", registered_model_name="ElasticnetModel")
            else:
                mlflow.sklearn.log_model(model, "model")
    


In [11]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation = ModelEvaluation(config=model_evaluation_config)
    model_evaluation.log_into_mlflow()
except Exception as e:
    raise e

[2026-02-03 10:10:48,478]: yaml file: e:\MLOPS\Data-Science-Project\config\config.yaml loaded successfully: common: yaml file: e:\MLOPS\Data-Science-Project\config\config.yaml loaded successfully]
[2026-02-03 10:10:48,482]: yaml file: e:\MLOPS\Data-Science-Project\config\params.yaml loaded successfully: common: yaml file: e:\MLOPS\Data-Science-Project\config\params.yaml loaded successfully]
[2026-02-03 10:10:48,487]: yaml file: e:\MLOPS\Data-Science-Project\config\schema.yaml loaded successfully: common: yaml file: e:\MLOPS\Data-Science-Project\config\schema.yaml loaded successfully]
[2026-02-03 10:10:48,489]: created directory at: artifacts: common: created directory at: artifacts]
[2026-02-03 10:10:48,491]: created directory at: artifacts/model_evaluation: common: created directory at: artifacts/model_evaluation]


e:\MLOPS\Data-Science-Project\venv\lib\site-packages\sklearn\base.py:442: InconsistentVersionWarning: Trying to unpickle estimator ElasticNet from version 1.5.1 when using version 1.7.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


[2026-02-03 10:10:50,739]: json file saved at: artifacts\model_evaluation\metrics.json: common: json file saved at: artifacts\model_evaluation\metrics.json]


2026/02/03 10:10:52 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
e:\MLOPS\Data-Science-Project\venv\lib\site-packages\mlflow\models\model.py:1209: FutureWarning: Saving scikit-learn models in the pickle or cloudpickle format requires exercising caution because these formats rely on Python's object serialization mechanism, which can execute arbitrary code during deserialization.The recommended safe alternative is the 'skops' format.
  flavor.save_model(path=local_path, mlflow_model=mlflow_model, **kwargs)
Successfully registered model 'ElasticnetModel'.
2026/02/03 10:11:12 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: ElasticnetModel, version 1
Created version '1' of model 'ElasticnetModel'.


🏃 View run zealous-moth-630 at: https://dagshub.com/saikishan2452003/Data-Science-Project.mlflow/#/experiments/0/runs/1ccbe521136d4164a4790da1391ef28d
🧪 View experiment at: https://dagshub.com/saikishan2452003/Data-Science-Project.mlflow/#/experiments/0
